# RDFLib

1. Uso de `rdflib` para generar un grafo RDF a partir de datos
2. Consulta y administración del grafo, tanto con SPARQL como con métodos propios de rdflib
3. Visualización del grafo mediante networkx

[https://rdflib.readthedocs.io/en/stable/index.html](https://rdflib.readthedocs.io/en/stable/index.html)


## Datos
Lo usual es partir de un fichero o de una consulta que devuelven un dataset. Para este ejemplo sencillo se parte de una pequeña lista de diccionarios.
### Datos iniciales

In [ ]:
pers = [ 
    {'nombre':'Marcos', 'año_nacimiento':'1962', 'lugar_nacimiento':'Salamanca', 'tiene_madre':'Belén', 'tiene_padre':'Ángel'},
    {'nombre':'Arturo', 'año_nacimiento':'1966', 'lugar_nacimiento':'Salamanca', 'tiene_madre':'Belén', 'tiene_padre':'Ángel'},
    {'nombre':'Pedro', 'año_nacimiento':'1979', 'lugar_nacimiento':'Badajoz', 'tiene_madre':'Inés', 'tiene_padre':'Pablo'},
    {'nombre':'Luis', 'año_nacimiento':'1984', 'lugar_nacimiento':'Madrid', 'tiene_madre':'Isabel', 'tiene_padre':'Jorge'},
    {'nombre':'Ana', 'año_nacimiento':'1982', 'lugar_nacimiento':'Madrid', 'tiene_madre':'Isabel', 'tiene_padre':'Jorge'},
    {'nombre':'Sonia', 'año_nacimiento':'1980', 'lugar_nacimiento':'Valencia', 'tiene_madre':'María', 'tiene_padre':'Fernando'},
    {'nombre':'Begoña', 'año_nacimiento':'2010', 'lugar_nacimiento':'Madrid', 'tiene_madre':'Sonia', 'tiene_padre':'Pedro'},
    {'nombre':'Marta', 'año_nacimiento':'2015', 'lugar_nacimiento':'Soria', 'tiene_madre':'Ana', 'tiene_padre':'Arturo'},
    {'nombre':'Juan', 'año_nacimiento':'2020', 'lugar_nacimiento':'Teruel', 'tiene_madre':'Ana', 'tiene_padre':'Pedro'} ]

print(f"{pers[0]['nombre']} nació en {pers[0]['año_nacimiento']} en {pers[0]['lugar_nacimiento']}")

### Datos reformateados como dataframe
Como los datos son tabulares, se presentan en un DataFrame, para facilitar su lectura.

In [ ]:
import pandas as pd

In [ ]:
pers_df = pd.DataFrame(pers)
display(pers_df)

print(f"{pers_df['nombre'][0]} nació en {pers_df['año_nacimiento'][0]} en {pers_df['lugar_nacimiento'][0]}")

## Grafo
Con los datos de la tabla anterior se generan tripletas (\<sujeto\> \<predicado\> \<objeto\>) como las siguientes:
+ \<Juan\> \<año_de_nacimiento\> "2020"
+ \<Juan\> \<tiene_madre\> \<Ana\>

In [ ]:
from rdflib import Graph, Literal, URIRef, Namespace, RDF, RDFS

### Espacios de nombres

In [ ]:
EXR = Namespace("http://example.org/recursos#")
EXP = Namespace("http://example.org/propiedades#")

# Recursos citados desde la lista de diccionarios 'pers'
print(URIRef(EXR + pers[0]['nombre']))
print(URIRef(EXP.año_nacimiento))
print(Literal(pers[0]['año_nacimiento']))

### Generación del grafo

In [ ]:
g = Graph()

for p in pers:
    sujeto = URIRef(EXR + p['nombre'])
    for key, value in p.items():
        if key=='nombre':
            pass
        elif key=='año_nacimiento' :
            g.add( (sujeto, URIRef(EXP + key), Literal(value)) )
        else:
            g.add( (sujeto, URIRef(EXP + key), URIRef(EXR + value)) )

In [ ]:
# Serialización del grafo RDF creado, en formato Turtle.
print(g.serialize(format='turtle'))

### Visualización del grafo inicial
Entre otras opciones posibles, se importa el paquete Jaal para visualizar interactivamente el grafo (en pestaña aparte). Descomente la primera línea, en caso de que no haya sido previamente instalado.

In [ ]:
#!pip install Jaal
from jaal import Jaal
from dash import html
import dash_html_components as html

Este paquete require un dataframe con al menos dos columnas ("from", "to") con los pares de nodos relacionados. La siguiente función genera ese dataframe desde el grafo RDF, mostrando sólo la parte final del URIRef de cada entidad.

In [ ]:
def rdf2edge_df(grafo):
    ternas = []
    for s, p, o in grafo:
        if isinstance(o,URIRef):
            oout = o.fragment
        else:
            oout = o
        lista_grafo.append([s.fragment,p.fragment,oout])
    return pd.DataFrame(ternas,columns=["from","label","to"])

edge_df = rdf2edge_df(g)
Jaal(edge_df).plot(directed=True)

## Consultas y actualización
RDFLib permite consultar y administrar el grafo de dos formas complementarias:
+ mediante el lenguaje estándar de consulta y administración SPARQL
+ mediante gestión directa y a bajo nivel de los objetos Python del grafo
### Consultas mediante SPARQL
Consulta SELECT: devuelve todas las coincidencias en el grafo del patrón declarado en WHERE. En este caso, cada persona de la que conste tanto su madre como su padre.

In [ ]:
consulta1='''
  PREFIX rec: <http://example.org/recursos#>
  PREFIX prop: <http://example.org/propiedades#>
  SELECT ?s ?m ?p
  WHERE {
    ?s prop:tiene_madre ?m ;
       prop:tiene_padre ?p .
  }
'''
resultados1 = g.query(consulta1)

for fila in resultados1:
    print(f"{fila.s}, madre: {fila.m}, padre: {fila.p}")

### Actualización mediante SPARQL
Actualización (UPDATE) del grafo: permite ejecutar INSERT o DELETE sobre entidades o tripletas seleccionadas en WHERE. En este caso, para cada instancia (?s,?m,?p) en el grafo del patrón en WHERE se asigna a cada una de sus componentes la clase Persona.

In [ ]:
consulta2='''
  PREFIX rec: <http://example.org/recursos#>
  PREFIX prop: <http://example.org/propiedades#>
  INSERT {
    rec:Persona rdf:type rdfs:Class .
    ?s rdf:type rec:Persona .
    ?m rdf:type rec:Persona .
    ?p rdf:type rec:Persona .
  }
  WHERE {
    ?s prop:tiene_madre ?m ;
       prop:tiene_padre ?p .
  }
'''
g.update(consulta2)

print(g.serialize(format='turtle'))

### Consulta mediante métodos Python
En este caso, se requieren todas las tripletas del grafo que tienen por propiedad 'lugar de nacimiento'.

In [ ]:
for s, p, o in g.triples( (None, EXP.lugar_nacimiento, None) ):
    print(s,p,o)

### Actualización mediante métodos Python
Se añade una terna, que declara la clase Ciudad. Y se inserta, para cada ciudad del grafo, su declaración como instancia de la clase Ciudad.

In [ ]:
g.add( (EXR.Ciudad, RDF.type, RDFS.Class) )

for s, p, o in g.triples( (None, EXP.lugar_nacimiento, None) ):
    g.add( (o, RDF.type, EXR.Ciudad) )

In [ ]:
# Comprobación: instancias de la clase Ciudad
for s, p, o in g.triples( (None, RDF.type, EXR.Ciudad) ):
    print(s,p,o)